In [ ]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import copy
import base64
import regex

import time

In [ ]:
IMG_PATH = "/content/drive/MyDrive/Master_AI/AIVP/Project/Data/mobile_images/"
LOAD_FILE_PATH = "/content/drive/MyDrive/Master_AI/AIVP/Project/Data/mobile_data_img.csv"
SAVE_FILE_PATH = "/content/drive/MyDrive/Master_AI/AIVP/Project/Data/md_wImages2.csv"

In [ ]:
brands = ['Apple','Samsung','Xiaomi','Huawei','Sony','Nokia',
            'Oppo','Vivo','LG','OnePlus','Realme','Infinix','Google',
            'ZTE','Asus','HTC','E-tel','Greentel','Lenovo','Dialog',
            'Sony Ericsson','Moto','Acer','BlackBerry','China Mobile',
            'Micromax','Ag-tel','Alcatel','iPro','Zigo','Microsoft']

def label_bid (row):
  for id,brand in enumerate(brands):
    if row['Brand'] == brand:
      return id
  return -1

def label_cdn (row):
  if row['Condition'] == 'New':
    return 0
  if row['Condition'] == 'Used':
    return 1
  return -1

def lable_usage(dataframe,val_freq = 10,test_freq=10):

  brand_counter = np.zeros((2, len(brands)))
  rdy2Val = np.zeros((2, len(brands)), dtype=bool)
  rdy2Tst = np.zeros((2, len(brands)), dtype=bool)


  for l_i in range(len(dataframe)):

    brand_counter[dataframe.at[l_i,'Condition_id'],dataframe.at[l_i,'Brand_id']] +=1
      
    if((brand_counter[dataframe.at[l_i,'Condition_id']][dataframe.at[l_i,'Brand_id']] % val_freq) == 0):
      rdy2Val[dataframe.at[l_i,'Condition_id']][dataframe.at[l_i,'Brand_id']] = True
    if((brand_counter[dataframe.at[l_i,'Condition_id']][dataframe.at[l_i,'Brand_id']] % test_freq) == 0):
      rdy2Tst[dataframe.at[l_i,'Condition_id']][dataframe.at[l_i,'Brand_id']] = True

    if(rdy2Val[dataframe.at[l_i,'Condition_id']][dataframe.at[l_i,'Brand_id']]):
      dataframe.at[l_i,'Usage'] = 'Validation'
      rdy2Val[dataframe.at[l_i,'Condition_id']][dataframe.at[l_i,'Brand_id']] = False
    elif(rdy2Tst[dataframe.at[l_i,'Condition_id']][dataframe.at[l_i,'Brand_id']]):
      dataframe.at[l_i,'Usage'] = 'Testing'
      rdy2Tst[dataframe.at[l_i,'Condition_id']][dataframe.at[l_i,'Brand_id']] = False
  
  return dataframe


def lable_usage_old(dataframe,val_freq = 10,test_freq=10):

  brand_counter_used = [0] * len(brands)
  brand_counter_new = [0] * len(brands)
  rdy2Val_used = [False] * len(brands)
  rdy2Tst_used = [False] * len(brands) 
  rdy2Val_new = [False] * len(brands)
  rdy2Tst_new = [False] * len(brands) 
  
  for l_i in range(len(dataframe)):
    for b_i, brand in enumerate(brands):
      if(dataframe.at[l_i,'Brand'] == brand):
        brand_counter[b_i] += 1
        if((brand_counter[b_i] % val_freq) == 0):
          rdy2Val[b_i] = True
        if((brand_counter[b_i] % test_freq) == 0):
          rdy2Tst[b_i] = True
        if(rdy2Val[b_i]):
          dataframe.at[l_i,'Usage'] = 'Validation'
          rdy2Val[b_i] = False
        elif(rdy2Tst[b_i]):
          dataframe.at[l_i,'Usage'] = 'Testing'
          rdy2Tst[b_i] = False
  return dataframe


def img2str(image):
  shape_a = image.shape
  flat_img = image.reshape((image.shape[0]*image.shape[1]*image.shape[2]))
  string = ""
  shape_str = ""
  for cell in shape_a:
    shape_str += (str(cell)+' ')
  for cell in flat_img:
    string += (str(cell)+' ')
  string = string[:-1]
  shape_str = shape_str[:-1]
  return string,shape_str

In [ ]:
def cut_img (image):
  shape = image.shape
  cl_1, cl_2 = 0, 0
  if shape[0] > 107 :
    cl_1 = int((shape[0]-107) / 2)
  if shape[1] > 143 :
    cl_2 = int((shape[1] - 143) / 2)
  if (cl_1>0) or (cl_2>0):
    image = image[cl_1:,cl_2:,:]
  return image[:100,:140,:]

In [ ]:
def load_img2df(dataframe):
  t0 = time.process_time ()
  t_cp = t0
  len_of_df = len(dataframe)
  for i in range(len(dataframe)):
    try:
      img = mpimg.imread(IMG_PATH+dataframe.iloc[i]["Image_File"])
      #print('0',end=' ')
      img = cut_img(img)
      #print('a',end=' ')
      simg,imgs = img2str(img)
      #print('b',end=' ')
      dataframe.at[i,'ImageShape'] = imgs
      #print('c',end=' ')
      dataframe.at[i,'ImageData'] = simg
      #print('d',end=' ')
    except:
      dataframe.at[i,'Condition_id']=-1
      print('fail at ' + str(i))
      continue
    tn = time.process_time()
    t1 = tn - t_cp
    if(t1 > 10):
      t_cp = tn
      print( str(i) + '/' + str(len_of_df) + ' in ' + str(int((tn-t0)/10)) + ' min')

  return dataframe

In [ ]:
print('--- Start Operation ---')
print('Load Dataframe', end = ' ')

df = pd.read_csv(LOAD_FILE_PATH)

print("[check]")
print('Expand Dataframe', end = ' ')

df = df.reset_index(drop=True)
df.insert(2,'Usage','Training')
df.insert(3,'Brand_id',-1)
df.insert(5,'Condition_id',-1)
df.insert(8,'ImageShape','')
df.insert(9,'ImageData','')

print("[check]")
print('Label Brand', end = ' ')

df['Brand_id'] = df.apply (lambda row: label_bid(row), axis=1)

print("[check]")
print('Label Condition', end = ' ')

df['Condition_id'] = df.apply (lambda row: label_cdn(row), axis=1)

print("[check]")
print('Added Images')

df = load_img2df(df)

print('Added Images [check]')
print('Clear Corrupted entries', end = ' ')

df.drop( df[ df['Condition_id'] == -1 ].index , inplace=True)
df = df.reset_index(drop=True)

print("[check]")
print('Label Usage', end = ' ')

df = lable_usage(df)

print("[check]")
print('Save Dataframe', end = ' ')

df.to_csv(SAVE_FILE_PATH)

print("[check]")
print('--- Operation Finished ---')

--- Start Operation ---
Load Dataframe [check]
Expand Dataframe [check]
Label Brand [check]
Label Condition [check]
Added Images
fail at 164
fail at 245
264/10200 in 1 min
534/10200 in 2 min
fail at 718
fail at 720
fail at 722
804/10200 in 3 min
fail at 969
fail at 976
fail at 977
fail at 978
1078/10200 in 4 min
1346/10200 in 5 min
fail at 1591
1608/10200 in 6 min
1875/10200 in 7 min
fail at 1943
fail at 1949
2147/10200 in 8 min
2416/10200 in 9 min
2685/10200 in 10 min
2950/10200 in 11 min
3216/10200 in 12 min
3475/10200 in 13 min
fail at 3502
fail at 3718
3742/10200 in 14 min
4000/10200 in 15 min
4260/10200 in 16 min
4526/10200 in 17 min
4792/10200 in 18 min
fail at 5051
fail at 5053
5058/10200 in 19 min
fail at 5101
5321/10200 in 20 min
fail at 5367
5588/10200 in 21 min
5852/10200 in 22 min
fail at 5961
fail at 5963
fail at 5964
fail at 5965
fail at 5966
fail at 5970
fail at 5974
fail at 5978
fail at 5980
fail at 5984
fail at 5987
fail at 5994
fail at 5995
6126/10200 in 23 min
fail a